In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Built-in Tools
[Tools List](https://python.langchain.com/docs/integrations/tools)

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke('top news in india today')

print(result) # full str

d:\Documents\NewCodePractice\Langchain-CX\lang_venv\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


2 days ago · Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. 2 days ago · Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18. Jul 13, 2025 · Stay informed with breaking news and latest news on politics, business, entertainment sports, science, technology along with news updates from around the world. 1 day ago · Get Top News of the day with latest top news headlines of today. Get updated with current top news stories from India and the world only on Zee News. 2 days ago · India opener Pratika Rawal was deemed to have breached ICC Code of Conduct after two separate incidents of physical contact with England players during the first ODI in …


In [5]:
from langchain_community.tools import ShellTool # lc-exp

terminal_tool = ShellTool()

result = terminal_tool.invoke('whoami')

print(result) # run in windows cmd

Executing command:
 whoami
winmachine\sk



d:\Documents\NewCodePractice\Langchain-CX\lang_venv\lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [7]:
search_tool.metadata

### Custom Tools - Create a tool

#### 1. using @tool decorator (EASIEST)

In [9]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool, Tool
from langchain_core.messages import HumanMessage

In [ ]:
# create function with typehints and docstring, add @tool decorator

@tool
def multiply(a: int, b: int) -> int:
    """
    For any two given number this tool returns their mathematical multiplication
    """
    return a*b

In [11]:
multiply.name

'multiply'

In [7]:
multiply.description

'For any two given number this tool returns their mathematical multiplication'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [9]:
multiply.args_schema.model_json_schema() # LLMs see this schema, Pydantic v2 style
# print(multiply.args_schema.schema())    # depricated

{'description': 'For any two given number this tool returns their mathematical multiplication',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [ ]:
# multiply(2,3) # not a simple function anymore, so will throw error
# multiply.run(2, 3) # may not run
# multiply.run({"a": 2, "b": 3}) # OR, as runnable, recommended ->
multiply.invoke({"a": 2, "b": 3})

6

#### 2. using StructuredTool and Pydantic (stricter)


In [11]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [ ]:
class MultiplyInput(BaseModel):
    a : int = Field(..., description="the first number to multiply")    # ... means required
    b : int = Field(..., description="the second number to multiply")

# print(MultiplyInput.model_json_schema())

In [13]:
def multiply_func(a:int, b:int) -> int:
    return a*b

In [14]:
multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name = "multiply",                      # default - func name
    description = "multiply two numbers",   # default - func docstring
    args_schema=MultiplyInput               # pydantic class
)

In [15]:
print(multiply_tool.name)
print(multiply_tool.description)

multiply
multiply two numbers


In [16]:
result = multiply_tool.invoke({'a':3, 'b':6})
print(result)

18


In [17]:
from langchain.tools.render import format_tool_to_openai_function

tool_schema = format_tool_to_openai_function(multiply_tool)
print(tool_schema)

{'name': 'multiply', 'description': 'multiply two numbers', 'parameters': {'properties': {'a': {'description': 'the first number to multiply', 'type': 'integer'}, 'b': {'description': 'the second number to multiply', 'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}


C:\Users\sk\AppData\Local\Temp\ipykernel_13788\2803519040.py:3: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  tool_schema = format_tool_to_openai_function(multiply_tool)


#### 3. using BaseTool class (FLEXIBLE)
(abstract base class for all tools)

In [18]:
from langchain.tools import BaseTool
from typing import Type

In [19]:
class MultiplyInput(BaseModel):
    a : int = Field(..., description="the first number to multiply")
    b : int = Field(..., description="the second number to multiply")

# print(MultiplyInput.model_json_schema())

In [20]:
class MultiplyTool(BaseTool):
    name: str = "multiply",                      # default - func name
    description: str = "multiply two numbers",   # default - func docstring
    args_schema: Type[BaseModel] = MultiplyInput # pydantic class

    def _run(self, a: int, b:int) -> int:        # only way to implement async -> def _arun
        return a*b

### Create toolkit (collection of tools)

In [21]:
from langchain_core.tools import tool

# custom tools
@tool
def add(a:int, b:int) -> int:
    """Add two numbers"""
    return a+b

@tool
def multiply(a:int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [ ]:
class MathToolKit:
    def get_tools(self): # lc internally calls this
        return [add,multiply]
    
toolkit = MathToolKit()
tools = toolkit.get_tools() # [StructuredTool(name='add', ), ...]

for tool in tools:
    print(tool.name,'=>', tool.description)

add => Add two numbers
multiply => Multiply two numbers


### Binding tool (function) with LLM

In [23]:
llm = ChatOpenAI(name='gpt-4o-nano')
llm

ChatOpenAI(name='gpt-4o-nano', client=<openai.resources.chat.completions.completions.Completions object at 0x0000025B23E1C670>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025B252F1030>, root_client=<openai.OpenAI object at 0x0000025B23E1E260>, root_async_client=<openai.AsyncOpenAI object at 0x0000025B252F0FA0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [ ]:
llm_with_tools = llm.bind_tools([multiply]) # OR, tools
llm_with_tools
# here LLM is suggested to use this tool and process input and then generate output
# LLM doesnt run the tool, tool execution is handled by langchain or human

RunnableBinding(bound=ChatOpenAI(name='gpt-4o-nano', client=<openai.resources.chat.completions.completions.Completions object at 0x0000025B23E1C670>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025B252F1030>, root_client=<openai.OpenAI object at 0x0000025B23E1E260>, root_async_client=<openai.AsyncOpenAI object at 0x0000025B252F0FA0>, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply two numbers', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

In [25]:
# invoke(call) the tool
llm_with_tools.invoke('Hi, how are you?') # for this query llm will not need the tool, so wont suggest the tool

AIMessage(content="Hello! I'm here and ready to help. What can I assist you with today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 51, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtyHY97qqefaEdd7xUcgdAfayErmZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4e31ceb8-4da1-4070-ba23-5cc18ab2baf4-0', usage_metadata={'input_tokens': 51, 'output_tokens': 19, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
result = llm_with_tools.invoke('can you give me the product of 11 with ten') # will suggest
result
# content is empty but in tool_calls variable it will list all related tools

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_avxYexKemVaJ7bXzgAzjTpch', 'function': {'arguments': '{"a": 11, "b": 10}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_kH6nYOdS0roMkGMUf7AMbUI3', 'function': {'arguments': '{"a": 10, "b": 11}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 56, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtyHZe3cetfin6HeWP6zVSRoTMRbx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--33ec4dae-687a-4666-931a-3d3f9a256e69-0', tool_calls=[{'name': 'multiply', 'args': {'a': 11, 'b': 10}, 'id': 'call_avxYexKemVaJ7bXzgAzjTpch', 'type': 'tool_call

In [27]:
llm_with_tools.invoke('can you multiply 11 with ten').tool_calls[0] # keys - name, args, id, type

{'name': 'multiply',
 'args': {'a': 11, 'b': 10},
 'id': 'call_WFV5tzknC91kQnRjVHXKUHEy',
 'type': 'tool_call'}

### finally, tools execution

In [28]:
# multiply.invoke(result.tool_calls[0].args) # only give op
multiply.invoke(result.tool_calls[0]) # will give full op wrapped with ToolMessage

ToolMessage(content='110', name='multiply', tool_call_id='call_avxYexKemVaJ7bXzgAzjTpch')

In [ ]:
# finalize all at once
query = HumanMessage("can you multiply 11 with ten")
msg = [query]
msg

[HumanMessage(content='can you multiply 11 with ten', additional_kwargs={}, response_metadata={})]

In [30]:
result = llm_with_tools.invoke(msg)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Snp0ku8JWIdD1VyeDPWgYVOu', 'function': {'arguments': '{"a": 11, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 52, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtyHbkjSJLy1XXTSQV7xC70OxydXQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--901a2903-9fec-4010-b684-f3555d1be27e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 11, 'b': 10}, 'id': 'call_Snp0ku8JWIdD1VyeDPWgYVOu', 'type': 'tool_call'}], usage_metadata={'input_tokens': 52, 'output_tokens': 32, 'total_tokens': 84, 'input_token_details': {'audio': 0, 'cache_read':

In [31]:
msg.append(result)
msg

[HumanMessage(content='can you multiply 11 with ten', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Snp0ku8JWIdD1VyeDPWgYVOu', 'function': {'arguments': '{"a": 11, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 52, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtyHbkjSJLy1XXTSQV7xC70OxydXQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--901a2903-9fec-4010-b684-f3555d1be27e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 11, 'b': 10}, 'id': 'call_Snp0ku8JWIdD1VyeDPWgYVOu', 'type': 'tool_call'}], usage_metadata={'input_tok

In [32]:
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='110', name='multiply', tool_call_id='call_Snp0ku8JWIdD1VyeDPWgYVOu')

In [33]:
msg.append(tool_result)
msg

[HumanMessage(content='can you multiply 11 with ten', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Snp0ku8JWIdD1VyeDPWgYVOu', 'function': {'arguments': '{"a": 11, "b": 10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 52, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtyHbkjSJLy1XXTSQV7xC70OxydXQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--901a2903-9fec-4010-b684-f3555d1be27e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 11, 'b': 10}, 'id': 'call_Snp0ku8JWIdD1VyeDPWgYVOu', 'type': 'tool_call'}], usage_metadata={'input_tok

In [34]:
# sending the full content
llm_with_tools.invoke(msg) #.content

AIMessage(content='The result of multiplying 11 with 10 is 110.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 77, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtyHczu7fR9GIYI59NxZRODZsV3DJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--07a366f3-60d9-4dad-9570-5c4a9b906f30-0', usage_metadata={'input_tokens': 77, 'output_tokens': 14, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})